In [123]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import pandas as pd
import os
from PIL import Image

In [124]:
# class base_resnet(nn.Module):
#     def __init__(self):
#         super(base_resnet, self).__init__()
#         self.model = models.resnet50(pretrained=True)  # 更改为ResNet18

#         # 修改最后一层
#         self.model.fc = nn.Sequential(
#             nn.Linear(512, 1024, bias=True),  # ResNet18的最后一层输入特征数量为512
#             nn.BatchNorm1d(1024),
#             nn.ReLU(),
#             nn.Linear(1024, 176, bias=True)
#         )
        
#         # 将其他层的参数设置为不需要更新
#         for param in self.model.parameters():
#             param.requires_grad = False

#         # 只训练最后两层
#         for param in self.model.layer4.parameters():
#             param.requires_grad = True
#         for param in self.model.fc.parameters():
#             param.requires_grad = True

#     def forward(self, x):
#         x = self.model(x) 
#         return x

class base_resnet(nn.Module):
    def __init__(self):
        super(base_resnet, self).__init__()
        self.model = models.resnet50(pretrained=True)
        # 修改最后一层
        self.model.fc = nn.Sequential(
            nn.Linear(2048, 1024, bias=True),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, 176, bias=True)
        )
        
        # 将其他层的参数设置为不需要更新
        for param in self.model.parameters():
            param.requires_grad = False

        # 只训练最后两层
        for param in self.model.layer4.parameters():
            param.requires_grad = True
        for param in self.model.fc.parameters():
            param.requires_grad = True
 
    def forward(self, x):
        x = self.model(x) 
        return x

In [125]:
# 数据路径
train_csv = '/kaggle/input/classify-leaves/train.csv'
test_csv = '/kaggle/input/classify-leaves/test.csv'  # 假设你有一个测试集的CSV文件
images_dir = '/kaggle/input/classify-leaves/'

train_data = pd.read_csv(train_csv)
test_data = pd.read_csv(test_csv) 

In [126]:
categories = pd.unique(train_data['label']).tolist()
categories.sort()

In [127]:
print(categories)

['abies_concolor', 'abies_nordmanniana', 'acer_campestre', 'acer_ginnala', 'acer_griseum', 'acer_negundo', 'acer_palmatum', 'acer_pensylvanicum', 'acer_platanoides', 'acer_pseudoplatanus', 'acer_rubrum', 'acer_saccharinum', 'acer_saccharum', 'aesculus_flava', 'aesculus_glabra', 'aesculus_hippocastamon', 'aesculus_pavi', 'ailanthus_altissima', 'albizia_julibrissin', 'amelanchier_arborea', 'amelanchier_canadensis', 'amelanchier_laevis', 'asimina_triloba', 'betula_alleghaniensis', 'betula_jacqemontii', 'betula_lenta', 'betula_nigra', 'betula_populifolia', 'broussonettia_papyrifera', 'carpinus_betulus', 'carpinus_caroliniana', 'carya_cordiformis', 'carya_glabra', 'carya_ovata', 'carya_tomentosa', 'castanea_dentata', 'catalpa_bignonioides', 'catalpa_speciosa', 'cedrus_atlantica', 'cedrus_deodara', 'cedrus_libani', 'celtis_occidentalis', 'celtis_tenuifolia', 'cercidiphyllum_japonicum', 'cercis_canadensis', 'chamaecyparis_pisifera', 'chamaecyparis_thyoides', 'chionanthus_retusus', 'chionanthu

In [128]:
train_data = pd.concat([train_data.drop('label', axis=1), pd.get_dummies(train_data['label'], prefix='label').astype('float64')], axis=1)
train_data.head()

,image,label_abies_concolor,label_abies_nordmanniana,label_acer_campestre,label_acer_ginnala,label_acer_griseum,label_acer_negundo,label_acer_palmatum,label_acer_pensylvanicum,label_acer_platanoides,...,label_tilia_europaea,label_tilia_tomentosa,label_tsuga_canadensis,label_ulmus_americana,label_ulmus_glabra,label_ulmus_parvifolia,label_ulmus_procera,label_ulmus_pumila,label_ulmus_rubra,label_zelkova_serrata
0,images/0.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,images/1.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,images/2.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,images/3.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,images/4.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [129]:
train_data.shape

(18353, 177)

In [130]:
class LeafDataset(Dataset):
    def __init__(self, data_frame, root_dir):
        self.data_frame = data_frame
        self.root_dir = root_dir
        self.transform = transforms.Compose([
            transforms.PILToTensor(),
            transforms.Lambda(lambda x: x / 255.0) 
        ])
        
    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 0])  # image path
        image = Image.open(img_name)
        label = self.data_frame.iloc[idx, 1:].tolist()
        
        image = self.transform(image)
        label = torch.tensor(label,dtype=torch.float32)
        
        return image, label

In [131]:
train_dataset = LeafDataset(train_data, images_dir)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
print(train_size, val_size)

14682 3671


In [132]:
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])
batch_size = 128

train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=True)

In [133]:
for X,y in train_loader:
    print(X)
    break

tensor([[[[0.8275, 0.8314, 0.8314,  ..., 0.8902, 0.8863, 0.8863],
          [0.8314, 0.8314, 0.8314,  ..., 0.8941, 0.8902, 0.8902],
          [0.8314, 0.8314, 0.8353,  ..., 0.8941, 0.8902, 0.8902],
          ...,
          [0.8471, 0.8471, 0.8471,  ..., 0.8471, 0.8471, 0.8471],
          [0.8471, 0.8471, 0.8471,  ..., 0.8431, 0.8431, 0.8431],
          [0.8471, 0.8471, 0.8471,  ..., 0.8431, 0.8431, 0.8431]],

         [[0.8784, 0.8824, 0.8824,  ..., 0.9059, 0.9020, 0.9020],
          [0.8824, 0.8824, 0.8824,  ..., 0.9098, 0.9059, 0.9059],
          [0.8824, 0.8824, 0.8863,  ..., 0.9098, 0.9059, 0.9059],
          ...,
          [0.8471, 0.8471, 0.8471,  ..., 0.8471, 0.8471, 0.8471],
          [0.8471, 0.8471, 0.8471,  ..., 0.8510, 0.8510, 0.8510],
          [0.8471, 0.8471, 0.8471,  ..., 0.8510, 0.8510, 0.8510]],

         [[0.8431, 0.8471, 0.8471,  ..., 0.9020, 0.8980, 0.8980],
          [0.8471, 0.8471, 0.8471,  ..., 0.9059, 0.9020, 0.9020],
          [0.8471, 0.8471, 0.8510,  ..., 0

In [134]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
model = base_resnet().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-2)

/home/aoijays/miniconda3/envs/torch/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/aoijays/miniconda3/envs/torch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [135]:
def training(dataloader, model, loss_fn, optimizer):
    
    model.train()

    loss, correct= 0.0, 0.0
    n = 0

    for X,y in dataloader:
    
        X,y = X.to(device), y.to(device)
        output = model(X)

        cur_loss = loss_fn(output, y)
        _, pred = torch.max(output, axis = 1)
        _, ypred = torch.max(y, axis = 1)
            
        optimizer.zero_grad()
        cur_loss.backward()
        optimizer.step()

        loss += cur_loss * len(y)
        correct += torch.sum(ypred==pred).item()
        n += len(y)

    return loss / n, correct / n

def val(dataloader, model, loss_fn):
    
    model.eval()

    with torch.no_grad():

        loss, correct= 0.0, 0.0
        n = 0 

        for X,y in dataloader:

            X,y = X.to(device), y.to(device)
            output = model(X)
            _, pred = torch.max(output, axis = 1)
            _, ypred = torch.max(y, axis = 1)
            
            cur_loss = loss_fn(output, y)
            correct += torch.sum(ypred==pred).item()
            loss += cur_loss * len(y)
            n += len(y)
            

            
    return loss/n, correct/n

In [137]:
epochs = 10
max_acc = 0

params = model.state_dict()

# for epoch in range(epochs):

#     loss, acc = training(train_loader, model, loss_fn, optimizer)
#     print(f'epoch {epoch + 1}/{epochs}: loss = {loss} acc = {acc}')
#     loss, acc = val(val_loader, model, loss_fn)
#     print(f'Validation: loss = {loss} acc = {acc}\n')


    # if acc > max_acc:
    #     max_acc = acc
    #     params = model.state_dict()
    #     print('saved local best')

In [138]:
model.load_state_dict(params)

<All keys matched successfully>

In [140]:
class LeafTestDataset(Dataset):
    def __init__(self, data_frame, root_dir):
        self.data_frame = data_frame
        self.root_dir = root_dir
        self.transform = transforms.Compose([
            transforms.PILToTensor(),
            transforms.Lambda(lambda x: x / 255.0) 
        ])
        
    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 0])  # image path
        image = Image.open(img_name)
        
        image = self.transform(image)
        
        return image

In [171]:
test_dataset = LeafTestDataset(test_data, images_dir)

In [202]:

from matplotlib import axis


res = []

with torch.no_grad():
    n = len(test_dataset)
    for i in range(n):
        x = test_dataset[i].reshape(1,3,224,224).cuda()
        output = model(x)
        _,pred = torch.max(output, axis = 1)
        res.append(categories[pred.item()])
        # if len(res) > 10 : break
        
df = test_data['image']

    

In [206]:
res_df = pd.DataFrame({'result': res})
combined_df = pd.concat([test_data, res_df], axis=1)

In [208]:
combined_df[:10]

,image,result
0,images/18353.jpg,ilex_opaca
1,images/18354.jpg,ilex_opaca
2,images/18355.jpg,ilex_opaca
3,images/18356.jpg,ilex_opaca
4,images/18357.jpg,ilex_opaca
5,images/18358.jpg,ptelea_trifoliata
6,images/18359.jpg,ilex_opaca
7,images/18360.jpg,ilex_opaca
8,images/18361.jpg,ilex_opaca
9,images/18362.jpg,ptelea_trifoliata
